https://github.com/alan-turing-institute/xpandas/blob/master/examples/ExampleUsage.ipynb

В этом блокноте представлены основные примеры использования пакета XPandas.

### Example dataset

In [1]:
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

import numpy as np
import pandas as pd
import os, sys
import requests

sys.path.insert(0, '..')

from xpandas.data_container import *
from xpandas.transformers import TimeSeriesTransformer, TimeSeriesWindowTransformer

Пример использует [открытый набор данных временного ряда](http://timeseriesclassification.com/Downloads/FordA.zip). 

*Примечание переводчика. Вместо файла FordA.csv, предполагаемого в примере, на ссылке содержится набор train, test файлов в форматах txt, arff и ts. При переводе для проверки кода из пары train, test файлов в формате txt был  собран FordA_2.csv. С учетом сказанного часть строк исходного кода исключена, а часть добавлена. Исключенные строки отмечены «###», добавленные - «###Добавлено».*

Первое, что нам нужно сделать, это прочитать данные. Используем  функцию urlopen из Python библиотеки urllib для загрузки набора данных и ограничим длину каждой серии данных.

In [2]:
###url = "http://localhost:8888/tree/Xpandas/FordA.zip"
series_offset = 505

In [3]:
###Добавлено
zipfile = ZipFile('FordA_2.zip')
lines = zipfile.open('FordA_2.csv').readlines()

In [4]:
###url = urlopen(url)
###zipfile = ZipFile(BytesIO(url.read()))
###lines = zipfile.open('FordA/FordA.csv').readlines()
lines = [l.decode('utf-8') for l in lines]
lines = lines[series_offset:]

``lines`` теперь является списком строк, представляющих временные ряды (последовательности) в формате с разделением запятыми, который можно преобразовать в числа с плавающей запятой.

In [5]:
lines = [list(map(float, l.split(','))) for l in lines]

In [6]:
lines[0][:10]

[0.91200522,
 0.92226619,
 0.86658681,
 0.76889267,
 0.63734618,
 0.48121473,
 0.27536994,
 0.01158427,
 -0.31442362,
 -0.68563769]

Преобразуем каждую вложенную строку в более удобный объект ``pandas.Series``.

In [7]:
lines = [pd.Series(l) for l in lines]

In [8]:
###Добавлено
### Удалены последние элементы, являющиеся в FordA_2.csv значениями целевой переменной. 
### Далее в исходном коде эти элементы будут созданы заново.

In [9]:
for l in range(len(lines[0])):
    del lines[l][500]

# XPandas: Структуры данных

### XSeries

``XSeries`` – это 1d (одномерный) контейнер данных, в котором могут храниться любые объекты.

Используя объекты ``pandas.Series``, можно инкапсулировать список ``lines`` в объект ``XSeries``. У объекта есть глобальный индекс рядов и подиндекс для каждого ``pandas.Series``.

In [10]:
X = XSeries(lines)

In [11]:
X.head()

0    0      0.912005
1      0.922266
2      0.86658...
1    0     -0.307891
1     -0.016875
2      0.28940...
2    0     -0.887613
1     -0.939291
2     -0.89332...
3    0     -0.824560
1     -0.231051
2      0.54142...
4    0      0.484706
1      0.367138
2      0.11493...
dtype: object

Видим тип данных объекта ``XSeries``, который содержит тип включенных объектов, в данном случае, ``pandas.Series``. Таким образом ``XSeries`` создается из ``pandas.Series``. В частности, ``X`` поддерживает все методы встроенного в него объекта ``pandas.Series``.

### XDataFrame

``XDataFrame`` – это абстрактный 2d (двумерный) контейнер, который основан на ``pandas.DataFrame`` и хранит объекты ``XSeries``.

Основная особенность ``XDataFrame`` – это столбцы ``XSeries``, которые могут содержать любой тип данных. Например, это может быть набор данных, состоящий из последовательностей, изображений, текста, простых чисел или даже настраиваемых объектов. В идеале мы хотели бы обрабатывать такие разные типы данных в едином контейнере двумерных данных, например, реализовать цепочку преобразований для создания простой 2d матрицы обучающих данных.

Следующие примеры иллюстрируют такой процесс ``XDataFrame``.
Пусть ``Y`` – вектор меток для каждой строки.

``XDataFrame`` is an abstract 2d container that is based on ``pandas.DataFrame`` and stores ``XSeries`` objects.

In [12]:
Y = np.random.binomial(1, 0.5, X.shape[0])
Y = XSeries(Y)

In [13]:
Y.head()

0    0
1    1
2    1
3    1
4    0
dtype: int32

In [14]:
df = XDataFrame({
    'X': X,
    'Y': Y
})

In [15]:
df.head()

,X,Y
0,0 0.912005 1 0.922266 2 0.86658...,0
1,0 -0.307891 1 -0.016875 2 0.28940...,1
2,0 -0.887613 1 -0.939291 2 -0.89332...,1
3,0 -0.824560 1 -0.231051 2 0.54142...,1
4,0 0.484706 1 0.367138 2 0.11493...,0


Добавим новый столбец в ``XDataFrame``

In [16]:
df['X_1'] = XSeries([
    pd.Series(np.random.normal(size=100))
    for _ in range(X.shape[0])
])

In [17]:
df.head()

,X,Y,X_1
0,0 0.912005 1 0.922266 2 0.86658...,0,0 0.633648 1 -2.208455 2 -2.434117 3...
1,0 -0.307891 1 -0.016875 2 0.28940...,1,0 -1.872062 1 1.312424 2 -0.495028 3...
2,0 -0.887613 1 -0.939291 2 -0.89332...,1,0 0.384492 1 -1.244785 2 -0.387186 3...
3,0 -0.824560 1 -0.231051 2 0.54142...,1,0 0.030931 1 -1.494072 2 -0.852727 3...
4,0 0.484706 1 0.367138 2 0.11493...,0,0 0.265030 1 1.249638 2 -1.662275 3...


# XPandas: Трансформеры

Основная цель данного проекта – помощь в решении обычной задачи науки о данных по извлечению признаков из некоторых сложных объектов (например, последовательностей) перед собственно машинным обучением.

Имея ``XSeries`` из ``pandas.Series`` можно было бы, к примеру, извлечь признаки из каждой последовательности. Именно здесь трансформеры играют жизненно важную роль.

Каждый объект ``Transformer`` поддерживает те же методы ``fit, transform`` что и [преобразователи scikit-learn](http://scikit-learn.org/stable/data_transforms.html).

Рассмотрим несколько примеров.

### TimeSeriesWindowTransformer

Этот преобразователь вычисляет скользящее среднее с заданным размером окна.

In [18]:
tr = TimeSeriesWindowTransformer(windows_size=5)
tr.fit(X)
transformed_series = tr.transform(X)

In [19]:
transformed_series.head()

0    4      0.821419
5      0.735261
6      0.60588...
1    4      0.279993
5      0.559200
6      0.81733...
2    4     -0.771043
5     -0.590733
6     -0.30311...
3    4      0.601324
5      1.314630
6      1.95854...
4    4      0.032147
5     -0.251946
6     -0.56853...
dtype: object

Конечно же, при windows_size = 5 первые 4 элемента – NaN.

In [20]:
transformed_series[0].head(10)

4     0.821419
5     0.735261
6     0.605882
7     0.434882
8     0.218218
9    -0.046378
10   -0.353945
11   -0.681472
12   -0.989739
13   -1.224680
dtype: float64

### TimeSeriesTransformer

Попробуем другой трансформер, наверное, самый распространенный. Он извлекает из каждой ``pandas.Series`` несколько количественных характеристик, таких как среднее, стандартное (среднеквадратичное) отклонение, квантили. Можно создать свой собственный список характеристик. В результате получаем объект ``XDataFrame``.

In [21]:
tr = TimeSeriesTransformer()
tr.fit(X)
transformed_series = tr.transform(X)

In [22]:
type(transformed_series)

xpandas.data_container.data_container.XDataFrame

In [23]:
transformed_series.head().iloc[:, :5]

,None_TimeSeriesTransformer_mean,None_TimeSeriesTransformer_std,None_TimeSeriesTransformer_max,None_TimeSeriesTransformer_min,None_TimeSeriesTransformer_median
0,5.120000e-11,1.0,2.900807,-4.224172,0.015260
1,1.666000e-10,1.0,2.257974,-2.750389,-0.076291
2,-6.162000e-10,1.0,2.315894,-2.626029,0.021814
3,1.147334e-09,1.0,2.988510,-2.325682,-0.031677
4,4.640000e-10,1.0,2.670425,-2.323600,0.116048


Также можем использовать трансформер TSFresh.

*Примечание переводчика. К сожалению не удалось исправить ошибку, возникающую при выполнении кода с TSFresh. Поэтому несколько строк отмечены ###*

In [24]:
###from xpandas.transformers import TsFreshSeriesTransformer

In [25]:
###tr = TsFreshSeriesTransformer()
###tr.fit(X.head())
###transformed_series = tr.transform(X.head())

In [26]:
###transformed_series.head().iloc[:, :3]

### Встроенный пользовательскийтрансформер

Можно создать встроенный трансформер ``CustomTransfomer`` как, например

In [27]:
from xpandas.transformers import XSeriesTransformer

In [28]:
my_awesome_transfomer = XSeriesTransformer(transform_function=lambda x: x.std())

In [29]:
my_awesome_transfomer.fit(X)

XSeriesTransformer(name='XSeriesTransformer',
                   transform_function=<function <lambda> at 0x000001B1AF91FF70>)

In [30]:
my_awesome_transfomer.transform(X).head(10)

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
5    1.0
6    1.0
7    1.0
8    1.0
9    1.0
dtype: float64

Если необходимо создать собственный трансформер со сложной логикой, исследуйте внутреннюю реализацию трансформеров.

## XDataFrame transformer

Чтобы преобразовать **XDataFrame** , необходимо указать логику преобразования для столбцов, которые должны быть преобразованы с помощью **XDataFrameTransformer**.

Конструктор **XDataFrameTransformer** получает на вход словарь соответствия (mapping dictionary) {col_name: XSeries transformer}.

Например, применим **TimeSeriesWindowTransformer** к столбцу X и **TimeSeriesTransformer** к столбцу X_1.

При применении преобразования к столбцу он заменяется на преобразованный.

In [31]:
from xpandas.transformers import XDataFrameTransformer

In [32]:
df_transformer = XDataFrameTransformer({
    'X': TimeSeriesWindowTransformer(windows_size=4),
    'X_1': TimeSeriesTransformer()
})

In [33]:
df_transformer.fit(df)

C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


XDataFrameTransformer(transformations={'X': [TimeSeriesWindowTransformer(windows_size=4)],
                                       'X_1': [TimeSeriesTransformer()]})

In [34]:
transformed_df = df_transformer.transform(df)

In [35]:
transformed_df.head().iloc[:, :5]

,X_TimeSeriesWindowTransformer,Y,X_1_TimeSeriesTransformer_mean,X_1_TimeSeriesTransformer_std,X_1_TimeSeriesTransformer_max
0,3 0.867438 4 0.798773 5 0.68851...,0,-0.057699,1.000694,2.110093
1,3 0.137673 4 0.426965 5 0.70321...,1,-0.029061,0.971750,2.309251
2,3 -0.860246 4 -0.741900 5 -0.50359...,1,-0.175531,1.014078,2.177514
3,3 0.214538 4 0.957794 5 1.70105...,1,0.037549,0.950775,2.408700
4,3 0.186939 4 -0.080993 5 -0.40671...,0,-0.084881,1.083376,2.591736


## Pipeline transformer

Что ж, преобразователи это хорошо, но возможно ли создавать [конвейеры](http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html), как в scikit-learn?

Конечно! Давайте рассмотрим пример, где объединяются ``TimeSeriesTransformer`` и ``TimeSeriesWindowTransformer`` в объединенный конвейер, используя ``PipeLineChain``.

Сначала посмотрим пример ``PipeLineChain`` с ``XSeries``, а затем с ``XDataFrame``.

In [36]:
from xpandas.transformers import PipeLineChain

In [37]:
chain = PipeLineChain([
    ('moving average trans', TimeSeriesWindowTransformer(windows_size=5)),
    ('extract features', TimeSeriesTransformer())
])
chain.fit(X)

C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


PipeLineChain(steps=[('moving average trans',
                      TimeSeriesWindowTransformer(windows_size=5)),
                     ('extract features', TimeSeriesTransformer())])

In [38]:
chain.get_params

<bound method Pipeline.get_params of PipeLineChain(steps=[('moving average trans',
                      TimeSeriesWindowTransformer(windows_size=5)),
                     ('extract features', TimeSeriesTransformer())])>

In [39]:
transformed_X = chain.transform(X)

In [40]:
transformed_X.head().iloc[:, :2]

,None_TimeSeriesWindowTransformer_TimeSeriesTransformer_mean,None_TimeSeriesWindowTransformer_TimeSeriesTransformer_std
0,0.002414,0.886138
1,-0.000379,0.905873
2,0.004289,0.933099
3,0.002512,0.882039
4,-0.004902,0.900216


Отлично! Попробуем добавить преобразователь из scikit-learn в ``PipeLineChain``. Например, применим PCA для transformed_X.

In [41]:
from sklearn.decomposition import PCA

In [42]:
chain = PipeLineChain([
    ('moving average trans', TimeSeriesWindowTransformer(windows_size=5)),
    ('extract features', TimeSeriesTransformer()),
    ('pca', PCA(n_components=5))
])
chain.fit(X)

C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


PipeLineChain(steps=[('moving average trans',
                      TimeSeriesWindowTransformer(windows_size=5)),
                     ('extract features', TimeSeriesTransformer()),
                     ('pca', PCA(n_components=5))])

In [43]:
transformed_X = chain.transform(X)

In [44]:
transformed_X.head()

,0,1,2,3,4
0,1.129352,-0.738972,-0.012033,0.074599,0.007025
1,-0.198267,-0.342079,-0.082518,0.001834,-0.031451
2,-0.228682,-0.209318,-0.046778,-0.107703,-0.007085
3,-0.290770,0.466469,0.002260,-0.008628,-0.026071
4,-0.343227,0.221251,0.085504,0.106187,0.109666


Займемся еще более интересными делами! Добавим оценщика (estimator) из scikit-learn в конце PipeLineChain.

In [45]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

Убедимся, что типы X_train и X_test – это XSeries.

In [47]:
print(type(X_train))
print(type(X_test))

<class 'xpandas.data_container.data_container.XSeries'>
<class 'xpandas.data_container.data_container.XSeries'>


In [48]:
chain = PipeLineChain([
    ('moving average trans', TimeSeriesWindowTransformer(windows_size=5)),
    ('extract features', TimeSeriesTransformer()),
    ('pca', PCA(n_components=5)),
    ('logit_regression', LogisticRegression())
    
])
chain = chain.fit(X_train, y_train)

In [49]:
prediction = chain.predict(X_test)

In [50]:
accuracy_score(y_test, prediction)

0.490036231884058

Теперь попробуем ``PipeLineChain`` с ``XDataFrameTransformer``.
Представим набор данных из столбцов признаков: пол (0 или 1), возраст (int), последовательность (pandas.Series), а также целевой переменной (0 или 1). Создадим, ``PipeLineChain`` который извлекает признаки из последовательностей, выполняет ``PCA`` для всего набора признаков, а затем выполняет LogitRegression.

In [51]:
n = 100

df_features = XDataFrame({
    'gender': XSeries(np.random.binomial(1, 0.7, n)),
    'age': XSeries(np.random.poisson(25, n)),
    'series': XSeries([
        pd.Series(np.random.normal(size=500))
    ] * n)
})

target = XSeries(np.random.binomial(1, 0.45, n))

In [52]:
features_transformer = XDataFrameTransformer({
    'series': TimeSeriesTransformer()
})

In [53]:
pipe_line = PipeLineChain([
    ('extract_from_series', features_transformer),
    ('pca', PCA(n_components=5)),
    ('logit_regression', LogisticRegression())
])

In [54]:
df_features_train, df_features_test, \
        y_train, y_test = train_test_split(df_features, target)

In [55]:
pipe_line.fit(df_features_train, y_train)

C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


PipeLineChain(steps=[('extract_from_series',
                      XDataFrameTransformer(transformations={'series': [TimeSeriesTransformer()]})),
                     ('pca', PCA(n_components=5)),
                     ('logit_regression', LogisticRegression())])

In [56]:
pipe_line.predict(df_features_test)

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0])